# Original CC

In [ ]:
#STL-10
import os
import argparse
import torch
import torchvision
import numpy as np
from modules import resnet, network, transform
from evaluation import evaluation
from torch.utils import data
import copy

In [ ]:
def inference(loader, model, device):
    model.eval()
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            c = model.forward_cluster(x)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#args
# general
seed= 42
workers= 1
dataset_dir= "./dataset"

# train options
batch_size= 256
image_size= 224
start_epoch= 0
epoch= 1000
dataset= "STL-10" # CIFAR-10 / CIFAR-100 / STL-10 / ImageNet-10 / ImageNet-dogs / tiny-ImageNet

# model options
# resnet= "ResNet34" # ResNet18 / ResNet34 / ResNet50
feature_dim= 128
model_path= "save/STL-10"
reload= False

# loss options
learning_rate= 0.0003
weight_decay= 0.
instance_temperature= 0.5
cluster_temperature= 1.0

### STL-10

In [ ]:
image_size = 224
train_dataset = torchvision.datasets.STL10(
    root='./dataset',
    split="train",
    download=True,
    transform=transform.Transforms(size=image_size).test_transform,
)
test_dataset = torchvision.datasets.STL10(
    root='./dataset',
    split="test",
    download=True,
    transform=transform.Transforms(size=image_size).test_transform,
)
dataset = data.ConcatDataset([train_dataset, test_dataset])
class_num = 10

In [ ]:
data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=256,
        shuffle=True,
        drop_last=True,
        num_workers=workers,
    )


In [ ]:
#place original CC model; here
cc_model = "./models/cif_stl_chec_1000.tar"

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()


In [ ]:
res = resnet.get_resnet("ResNet34")
model = network.Network(res, feature_dim, class_num)
# model_fp = os.path.join(model_path, "checkpoint_{}.tar".format(start_epoch))
model.load_state_dict(torch.load(cc_model, map_location=device.type)['net'])
model.to(device)

In [ ]:
print("### Creating features from model ###")
X, Y = inference(data_loader, model, device)

In [ ]:
nmi, ari, f, acc = evaluation.evaluate(Y, X) #Y = Label, X = pred
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

## GAN attack

In [ ]:
from gan_attack import GAN_Attack

In [ ]:
use_cuda=True
image_nc=3
epochs = 60
batch_size = 128
BOX_MIN = 0
BOX_MAX = 1
model_num_labels = 10
device = 'cuda'

In [ ]:
GAN = GAN_Attack(device, model, model_num_labels, image_nc, BOX_MIN, BOX_MAX, "STL-10")
# model.device


In [ ]:
torch.cuda.empty_cache()

train_dataset = torchvision.datasets.STL10(
    root='./dataset',
    split="train",
    download=True,
    transform=transform.Transforms(size=image_size, s=0.5),
)
test_dataset = torchvision.datasets.STL10(
    root='./dataset',
    split="test",
    download=True,
    transform=transform.Transforms(size=image_size, s = 0.5),
)

dataset = data.ConcatDataset([train_dataset, test_dataset])

class_num = 10
data_loader2 = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

In [ ]:
torch.cuda.empty_cache()
advGAN.train(data_loader2, 120)

# Testing attack

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = './models/netG_cc_STL-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
def plot_examples(images, labels):
    print(type(images[0]), type(labels))
    print(images.shape)
    w = 10
    h = 10
    fig = plt.figure(figsize=(10, 20))
    columns = 11
    rows = 12
    for i in range(10):
#         img = np.random.randint(10, size=(h,w))
        fig.add_subplot(rows, columns, i+1)
#         img = images[i] / 2 + 0.5   # unnormalize
        img = images[i]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        
        plt.imshow(np.transpose(npimg, (1, 2, 0))) 
#         plt.imshow(npimg.astype('uint8'))
#   plt.show()

#         plt.imshow(np.transpose(np.reshape(images[i].detach().cpu(), (3, 32,32)), (1,2,0)))
        plt.title('#{}: {}'.format(i, labels[i]))
    plt.show()
    

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
def inference2(loader, model, device, clamp=0.03):
    model.eval()
    feature_vector = []
    labels_vector = []
    j = 1
    
    for step, (x, y) in enumerate(loader):
        torch.cuda.empty_cache()
        x = x.to(device)
        perturbation = pretrained_G(x)
        perturbation = torch.clamp(perturbation, -clamp, clamp)
        adv_img = perturbation + x
        adv_img = torch.clamp(adv_img, 0, 1)
        if j:
            torch.cuda.empty_cache()
            
            plot_examples(x,y)
            torch.cuda.empty_cache()
            plot_examples(adv_img, y)
            j = 0
        with torch.no_grad():
            c = model.forward_cluster(adv_img)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

In [ ]:
count_imgs =  1000
def save_examples(images, labels, noise=False, bno=0, adv=False, orig=False):
    print(type(images[0]), type(labels))
    global count_imgs
    
    if count_imgs <=0 :
        return
    for i in range(min(len(images), 20)):
        img = images[i]
        npimg = img.cpu().detach().numpy()   # convert from tensor
        npimg = np.clip(npimg, 0, 1)
        count_imgs -= 1
        if orig:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'Images/S10/CC/orig/CC_s10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if adv:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'Images/S10/CC/adv/CC_s10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if noise:
            npimg = npimg / 2 + 0.5 
            plt.imsave(f'Images/S10/CC/noise/CC_s10_b{bno}_{i}_noise_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
#     plt.show()
    

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
def inference_save(loader, model, device, pretrained_G):
    model.eval()
    feature_vector = []
    labels_vector = []
    j = 1
    
    for step, (x, y) in enumerate(loader):
        torch.cuda.empty_cache()
        x = x.to(device)
        perturbation = pretrained_G(x)
        perturbation = torch.clamp(perturbation, -0.1, 0.1)
        adv_img = perturbation + x
        adv_img = torch.clamp(adv_img, 0, 1)
#         if j:
#             torch.cuda.empty_cache()
            
#             plot_examples(x,y)
#             torch.cuda.empty_cache()
#             plot_examples(adv_img, y)
#             j = 0
        with torch.no_grad():
            c = model.forward_cluster(adv_img)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
        save_examples(x, y, bno=step, orig=True)
        save_examples(adv_img, c.cpu().detach().numpy(), bno=step, adv=True)
        save_examples(perturbation, c.cpu().detach().numpy(), bno=step, noise=True)
        
        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

In [ ]:
# to save samples
# inference_save(data_loader, model, 'cuda', pretrained_G)

In [ ]:
X, Y = inference2(data_loader, model, 'cuda', clamp=0.1)
nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from scipy.misc import toimage

# x = normalized_img * STD[:, None, None] + MEAN[:, None, None]

def pp(images, labels):
    print(type(images[0]), type(labels))
    print(images.shape)
#     w = 
#     h = 10
    fig = plt.figure(figsize=(15, 15))
    columns = 11
    rows = 12
    for i in range(35):
        fig.add_subplot(rows, columns, i+1)
        img = images[i]
#         img = img / 2 + 0.5
#         img = img / 2 + 0.5   # unnormalize
#         img = img * STD[:, None, None] + MEAN[:, None, None]
        img = img.detach().cpu().numpy()
        img = np.clip(img, 0, 1)
#         img = (img * 255).astype(np.uint8)
#         img = img / 2 + 0.5
#         img = img / 2 + 0.5 
#         npimg = img.detach().cpu().numpy()   # convert from tensor
        
#         plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
        plt.imshow(img.transpose(1, 2, 0))
#         plt.imshow(npimg)
#         plt.title('#{}: {}'.format(i, labels[i]))
    plt.show()

# Transferability

In [ ]:
#Base MICE

In [ ]:


import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = '../Generator_Models/STL10/netG_MICE_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

X, Y = inference2(data_loader, model, 'cuda', 0.4)
nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

In [ ]:
#Base NNM

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = '../Generator_Models/STL10/netG_NNM_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

X, Y = inference2(data_loader, model, 'cuda', 0.02)
nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

In [ ]:
#Base SCAN

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = '../Generator_Models/STL10/netG_SCAN_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

X, Y = inference2(data_loader, model, 'cuda', 0.05)
nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

In [ ]:
#Base RUC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = '../Generator_Models/STL10/netG_RUC_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

X, Y = inference2(data_loader, model, 'cuda', 0.05)
nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

In [ ]:
#Base SPICE

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = '../Generator_Models/STL10/netG_SPICE_stl10.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

X, Y = inference2(data_loader, model, 'cuda', 0.05)
nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

In [ ]:
#perturb

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
def inference_n(loader, model, device, clamp=0.05):
    model.eval()
    feature_vector = []
    labels_vector = []
    j = 1
    perturb_norm = 0.0
#     p_n = []
    
    for step, (x, y) in enumerate(loader):
        gc.collect()
        torch.cuda.empty_cache()
        x = x.to(device)
        perturbation = pretrained_G(x)
        perturbation = torch.clamp(perturbation, -clamp, clamp)
        perturb_norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
        
        
        adv_img = perturbation + x
        adv_img = torch.clamp(adv_img, 0, 1)
        
        with torch.no_grad():
            c = model.forward_cluster(adv_img)

        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
    
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector, perturb_norm/len(loader)

# Extra experiements

In [ ]:
#Original Model

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
pretrained_generator_path = './models/netG_cc_STL-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', j) #0.001 -> 10

    print(f'clamp {j} avg norm: {norm}')

    nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
# plt.plot(x, y, label = "line 1", linestyle="-")
# plt.plot(y, x, label = "line 2", linestyle="--")
# plt.plot(x, np.sin(x), label = "curve 1", linestyle="-.")
# plt.plot(x, np.cos(x), label = "curve 2", linestyle=":")

plt.plot(norm_l, nmi_l, label = "nmi", linestyle="-")
plt.plot(norm_l, ari_l, label = "ari", linestyle="-")
plt.plot(norm_l, acc_l, label = "acc", linestyle="-")
plt.xlabel("Perturbation Norm")
plt.ylabel("Performace")
plt.legend()
plt.savefig('cc_cifar10.png')
plt.show()

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
#same acc form

In [ ]:
import eval_cus

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
import eval_cus
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
# clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4]
clamp = [0, 0.03, 1]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', j)
    print(norm)
    
    Y = torch.from_numpy(Y).cuda()    
    X = torch.from_numpy(X).cuda()
    class_names =  ['airplane',
                 'bird',
                 'car',
                 'cat',
                 'deer',
                 'dog',
                 'horse',
                 'monkey',
                 'ship',
                 'truck']
    clustering_stats_adv = eval_cus.check(Y, X, 10, class_names, compute_confusion_matrix=True, confusion_matrix_file=None, output_file2=f"CC_s10_{j}_{norm}.pdf")
    acc = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    print(f'clamp {j} avg norm: {norm}')

#     nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
import eval_cus
import importlib
importlib.reload(eval_cus)

In [ ]:
class_names =  ['airplane',
             'bird',
             'car',
             'cat',
             'deer',
             'dog',
             'horse',
             'monkey',
             'ship',
             'truck']
X, Y, norm = inference_n(data_loader, model, 'cuda', 0)
print(norm)

Y = torch.from_numpy(Y).cuda()    
X = torch.from_numpy(X).cuda()
clustering_stats_adv = eval_cus.check(Y, X, 10, class_names, compute_confusion_matrix=True, output_file2= 'cc_0.1_orig.pdf')

acc = clustering_stats_adv['ACC']
nmi = clustering_stats_adv['NMI']
ari = clustering_stats_adv['ARI']
print(f'clamp 0.03, avg norm: {norm}')

#     nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)